# SetFit pipeline

# Dependencies

In [1]:
# !pip install setfit
# !pip install datasets
%load_ext autoreload
%autoreload 2

In [2]:
# from google.colab import drive
# import pandas as pd

# drive.mount('/content/drive')

In [3]:
import pandas as pd
import datasets
from datasets import Dataset
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer, sample_dataset
import evaluation_metrics
import gc
import zeroberto
tqdm.pandas()


# Data Prep

In [5]:
use_zeroshot_previous_step = True
exec_time = 'first_test'

which_dataset = 'folhauol'
# which_dataset = 'bbc-news'

split = "zeroshot" if use_zeroshot_previous_step else "fewshot"
n = 4
test_dataset_sample_size = 1/32
## 1/64 ---> <6 min
## 1/32 ---> <12 min
## 1/16 ---> <22 min
random_state = 422

In [46]:
top_n = n

## Import Data

In [21]:
if which_dataset=='folhauol':
  arq = '/Users/alealcoforado/Documents/Projetos/Datasets/folhauol/folhauol_clean_df_articles.csv'
  # arq = '/content/drive/MyDrive/folhauol_clean_df_articles.csv'
  raw_data = pd.read_csv(arq)
  raw_data['full_text'] = raw_data['title'].astype(str)+raw_data['text'].astype(str)
  data_col = 'full_text'
  class_col = 'category'

if which_dataset == 'bbc-news':
  #o dataset bbc-news não tem coluna de título 
  # arq = '/content/drive/MyDrive/zeroshot_new_paper/bbc_news_train_plus_test.csv'
  arq = '/Users/alealcoforado/Documents/Projetos/Datasets/bbc-news/BBC News Train.csv'
  raw_data = pd.read_csv(arq, sep = ',')
  data_col = 'Text'
  class_col = 'Category'

### Data from Zero-Shot previous step

In [22]:
zeroshot_data_local_path = '/Users/alealcoforado/Documents/Projetos/Datasets/{which_dataset}/'.format(which_dataset=which_dataset)
zeroshot_preds_and_probs_file = 'predictions_and_probabilities_test_{exec_time}.csv'.format(exec_time=exec_time)
preds_probs_df = pd.read_csv(zeroshot_data_local_path+zeroshot_preds_and_probs_file)

# zeroshot_config_file = 'zeroshot_config_test_{exec_time}.csv'.format(exec_time=exec_time)
# config_df = pd.read_csv(zeroshot_data_local_path+zeroshot_config_file)

In [23]:
def getZeroshotPreviousData(df_results_zeroberto,top_n = 8):
    df_results_zeroberto.index = df_results_zeroberto['Unnamed: 0'] ### recover original indexes for dataset
    df_top_n = df_results_zeroberto.sort_values(['top_probability','prediction'], ascending=False).groupby('prediction').head(top_n)
    df_top_n = df_top_n.drop(columns=["Unnamed: 0",class_col,class_col+"_code"])
    return df_top_n


In [41]:
if use_zeroshot_previous_step == True:
    zeroshot_previous_data = getZeroshotPreviousData(preds_probs_df)
    raw_data_final = raw_data.join(zeroshot_previous_data)
#     ## overwrite true labels with predictions from zeroshot
    new_class_col = 'new_'+class_col
    print(new_class_col)
    raw_data_final.loc[~raw_data_final['prediction'].isna(),new_class_col] = raw_data_final['prediction'] 
    
#     ## keep true labels of the rest, for testing
    raw_data_final.loc[raw_data_final['prediction'].isna(),new_class_col] = raw_data_final[class_col]
if use_zeroshot_previous_step == False:
    raw_data_final = raw_data
    raw_data_final[new_class_col] = raw_data_final[class_col]

# raw_data_final[new_class_col] = raw_data_final[new_class_col].map(dict_classes_folha)

new_category


In [42]:
zeroshot_previous_data.shape

(76, 4)

In [51]:
df_encoded = evaluation_metrics.Encoder(raw_data_final,[new_class_col])
df_encoded[~df_encoded['prediction'].isna()].groupby(new_class_col)[[data_col,new_class_col]].apply(lambda s: s.sample(2,random_state=random_state))

ValueError: Cannot take a larger sample than population when 'replace=False'

## Dataset Split

### Standard SetFit (Few-Shot)

In [44]:
### divide dataframe em treino e teste, com n samples para few-shot

if (split=="fewshot"):
  df_train = df_encoded.groupby('new_Category_code')[['full_text','new_Category_code']].apply(lambda s: s.sample(min(len(s),n)))
  keys = list(df_train.columns.values)
  i1 = df_encoded.set_index(keys).index
  i2 = df_train.set_index(keys).index
  df_test = df_encoded[~i1.isin(i2)]
  df_test = df_test.groupby('new_Category_code')[['full_text','new_Category_code']].apply(lambda x:x.sample(int(len(x)*test_dataset_sample_size)))
  df_train = df_train.astype(str)
  df_test = df_test.astype(str)

### Zeroberto SetFit

In [52]:
if (split == "zeroshot"):
  df_train = df_encoded[~df_encoded['prediction'].isna()].groupby(new_class_col)[[data_col,new_class_col]].apply(lambda s: s.sample(min(len(s),top_n),random_state=random_state))

  keys = list(df_train.columns.values)
  i1 = df_encoded.set_index(keys).index
  i2 = df_train.set_index(keys).index
  df_test = df_encoded[~i1.isin(i2)]
  df_test = df_test.groupby(new_class_col)[[data_col,new_class_col]].apply(lambda x:x.sample(int(len(x)*test_dataset_sample_size),random_state=random_state))
  df_train = df_train.astype(str)
  df_test = df_test.astype(str)

### Build Dataset Dict

In [53]:
### transforma dataframes em datasetdict

train_dataset = Dataset.from_dict(df_train)
test_dataset = Dataset.from_dict(df_test)
dataset_dict = datasets.DatasetDict({"train":train_dataset,"test":test_dataset})
dataset = dataset_dict
dataset

DatasetDict({
    train: Dataset({
        features: ['full_text', 'new_category'],
        num_rows: 43
    })
    test: Dataset({
        features: ['full_text', 'new_category'],
        num_rows: 2873
    })
})

# SetFit

## Model

In [56]:
model = saved_model

In [496]:
model_name = "ricardo-filho/bert-base-portuguese-cased-nli-assin-2"

#### Models
# "sentence-transformers/paraphrase-mpnet-base-v2"
# "ricardo-filho/bert-base-portuguese-cased-nli-assin-2"
# "ricardo-filho/bert-portuguese-cased-nli-assin-2"
# "neuralmind/bert-large-portuguese-cased"
# "joeddav/xlm-roberta-large-xnli"
# "openai-gpt"

model = SetFitModel.from_pretrained(model_name)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


## Parameters

In [57]:
%%time

batch_size = 8
num_text_pairs = 10
num_epochs = 1

setfit_config = {
    "model":model_name,
    "dataset":which_dataset,
    "batch_size":batch_size,
    "num_pairs":num_text_pairs,
    "num_epochs":num_epochs,
    "dataset_sample_size":test_dataset_sample_size,
    "N_examples":n,
    "split":split,
    "0shot_data":exec_time
}

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=setfit_config["batch_size"],
    num_iterations=setfit_config["num_pairs"], # Number of text pairs to generate for contrastive learning
    num_epochs=setfit_config["num_epochs"], # Number of epochs to use for contrastive learning
    column_mapping = {data_col: "text", new_class_col: "label"} # NÃO mudar
)

print(setfit_config)

{'model': '/Users/alealcoforado/Documents/Projetos/Modelos/setfit_top_n=4_n=8_15pairs_v1', 'dataset': 'folhauol', 'batch_size': 8, 'num_pairs': 10, 'num_epochs': 1, 'dataset_sample_size': 0.03125, 'N_examples': 4, 'split': 'zeroshot', '0shot_data': 'first_test'}
CPU times: user 139 µs, sys: 157 µs, total: 296 µs
Wall time: 266 µs


## Training

In [58]:
%%time
# trainer.train()
gc.collect()

CPU times: user 109 ms, sys: 156 ms, total: 265 ms
Wall time: 382 ms


208

## Inference

In [503]:
%%time
y_pred = zeroberto.getPredictions(trainer)
#### folhauol: aprox. 4 predicoes por segundo
#### bbcnews: aprox. 4 preds por segundo

CPU times: user 1h 34s, sys: 15min 15s, total: 1h 15min 49s
Wall time: 22min 22s


# Evaluation

## Metrics

In [ ]:
all_metrics = evaluation_metrics.get_metrics(y_pred ,test_dataset[new_class_col])
print(all_metrics)

{'weighted': [{'accuracy': 0.015315001740341106}, {'precision': 0.04790430994811901}, {'recall': 0.015315001740341106}, {'f1': 0.01980755741551899}], 'macro': [{'accuracy': 0.015315001740341106}, {'precision': 0.023423116910519695}, {'recall': 0.01463469403062753}, {'f1': 0.013475393598413229}]}


## Save Results

In [443]:
setfit_exec_time  = evaluation_metrics.saveResults(setfit_config,all_metrics)
print(setfit_config)

metrics_setfit_2023_02_10__17_53_56.csv
config_setfit_2023_02_10__17_53_56.csv
{'model': 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2', 'dataset': 'folhauol', 'batch_size': 8, 'num_pairs': 10, 'num_epochs': 1, 'dataset_sample_size': 0.0625, 'N_examples': 4, 'split': 'zeroshot', '0shot_data': '2023_02_10__09_58_13'}


## Save Model

In [445]:
model_path = "/Users/alealcoforado/Documents/Projetos/Modelos/{exec}".format(exec=setfit_exec_time)
print(model_path)
trainer.model._save_pretrained(save_directory=model_path)

/Users/alealcoforado/Documents/Projetos/Modelos/2023_02_10__17_53_56


## Load Model

In [55]:
load_path = r'/Users/alealcoforado/Documents/Projetos/Modelos/setfit_top_n=4_n=8_15pairs_v1'

saved_model = SetFitModel._from_pretrained(load_path)
model_name = load_path

/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


{'weighted': [{'accuracy': 0.7208791208791209},
  {'precision': 0.7966533533828771},
  {'recall': 0.7208791208791209},
  {'f1': 0.7383634133416869}],
 'macro': [{'accuracy': 0.7208791208791209},
  {'precision': 0.47294791260373964},
  {'recall': 0.6956977371368128},
  {'f1': 0.5148314501574386}]}

  {'weighted': [{'accuracy': 0.7088331515812432},
  {'precision': 0.8112290505989518},
  {'recall': 0.7088331515812432},
  {'f1': 0.7416526123050681}],
 'macro': [{'accuracy': 0.7088331515812432},
  {'precision': 0.5086466490124514},
  {'recall': 0.7260511792805407},
  {'f1': 0.5487853669870654}]}

1/4 dataset
  {'weighted': [{'accuracy': 0.712040293517433},
  {'precision': 0.7969596487586078},
  {'recall': 0.712040293517433},
  {'f1': 0.7365853189562236}],
 'macro': [{'accuracy': 0.712040293517433},
  {'precision': 0.4964842305418267},
  {'recall': 0.7028323441240695},
  {'f1': 0.5374745594204607}]}

full dataset
{'weighted': [{'accuracy': 0.7090563785338325},
  {'precision': 0.7967113724849181},
  {'recall': 0.7090563785338325},
  {'f1': 0.734316306685027}],
 'macro': [{'accuracy': 0.7090563785338325},
  {'precision': 0.4954781686924334},
  {'recall': 0.7020811904269998},
  {'f1': 0.5360406756304892}]}